In [1]:
file = open('cooking.stackexchange.txt', 'r')
# read all text
text = file.readlines()
# close the file
file.close()

In [2]:

sentences=[]
labels=[]
lab=[]
text[0]

'__label__sauce __label__cheese How much does potato starch affect a cheese sauce recipe?\n'

In [3]:
p=text[0].strip().split("__label__")
print(p)
p.pop(0)
k=p.pop()
k=k.split(" ")
m=k.pop(0)
p.append(m)
print(p,k,m)

['', 'sauce ', 'cheese How much does potato starch affect a cheese sauce recipe?']
['sauce ', 'cheese'] ['How', 'much', 'does', 'potato', 'starch', 'affect', 'a', 'cheese', 'sauce', 'recipe?'] cheese


In [4]:
from tqdm import tqdm
for i in tqdm(text):
    p=i.strip().split("__label__")
    p.pop(0)
    k=p.pop()
    k=k.split(" ")
    m=k.pop(0)
    p.append(m)
    s=''
    for i in range(len(p)):
        p[i]=p[i].strip()
        lab.append(p[i])
    sentences.append(s.join([i.lower()+" " for i in k]))
    labels.append(p)

100%|██████████| 15404/15404 [00:00<00:00, 204630.00it/s]


In [5]:
sentences

['how much does potato starch affect a cheese sauce recipe? ',
 'dangerous pathogens capable of growing in acidic environments ',
 'how do i cover up the white spots on my cast iron stove? ',
 'michelin three star restaurant; but if the chef is not there ',
 'without knife skills, how can i quickly and accurately dice vegetables? ',
 "what's the purpose of a bread box? ",
 'how to seperate peanut oil from roasted peanuts at home? ',
 'american equivalent for british chocolate terms ',
 'fan bake vs bake ',
 'regulation and balancing of readymade packed mayonnaise and other sauces ',
 'what kind of tea do you boil for 45minutes? ',
 'how long can batter sit before chemical leaveners lose their power? ',
 'can i re-freeze chicken soup after it has thawed? ',
 'ziploc vacuumed bags expand in sous vide ',
 'what can i use instead of corn syrup? ',
 'does soaking almonds have the same effect as blanching and removing the skins when making almond milk? ',
 'cake sinks in the middle when baki

In [6]:
import numpy as np
lab=np.unique(lab)

In [7]:
print(len(lab))

736


In [8]:
import nltk
from nltk.corpus import stopwords

In [9]:
import pandas as pd
import numpy as np

In [10]:
labels=np.array(labels)
sentences=np.array(sentences)

C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


In [11]:
labels

array([list(['sauce', 'cheese']), list(['food-safety', 'acidity']),
       list(['cast-iron', 'stove']), ..., list(['vegetables']),
       list(['culinary-uses', 'seeds']), list(['culinary-uses', 'herbs'])],
      dtype=object)

In [12]:
def text_process(mess):
    nopunc = [char for char in mess if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    return [word.lower() for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [13]:
import string
for i in tqdm(range(len(sentences))):
    sentences[i]=' '.join(text_process(sentences[i]))

100%|██████████| 15404/15404 [00:28<00:00, 534.81it/s]


In [14]:
len(sentences)

15404

In [15]:
#from sklearn.feature_extraction.text import CountVectorizer

In [16]:
d={}
idx=0
for j in labels:
    for i in j:
        if i not in d.keys():
            d[i]=idx
            idx+=1
        else:
            continue
print(len(d.keys()))

736


In [17]:
len(labels)

15404

In [18]:
wordfreq={}
for sent in sentences:
    token=nltk.word_tokenize(sent)
    for t in token:
        if t in wordfreq.keys():
            wordfreq[t]+=1
        else:
            wordfreq[t]=1
print(len(wordfreq.keys()))

9470


In [19]:
from tqdm import tqdm
import time
bowvec=[]
for sent in tqdm(sentences):
    token=nltk.word_tokenize(sent)
    sentbowvec=[]
    for t in list(wordfreq.keys()):
        if t in token:
            sentbowvec.append(token.count(t))
        else:
            sentbowvec.append(0)
    bowvec.append(sentbowvec)

100%|██████████| 15404/15404 [01:06<00:00, 232.06it/s]


In [20]:
bowvec=np.array(bowvec)

In [21]:
bowvec.shape

(15404, 9470)

In [22]:
lab_onehot=np.zeros([len(labels),736])

In [23]:
for i in tqdm(range(len(labels))):
    for k in labels[i]:
        lab_onehot[i][d[k]]=1

100%|██████████| 15404/15404 [00:00<00:00, 134090.71it/s]


In [24]:
lab_onehot.shape

(15404, 736)

In [25]:
from sklearn import preprocessing
bowvec=preprocessing.normalize(bowvec)

In [26]:
bowvec=bowvec.astype(np.float32)

In [27]:
from sklearn.decomposition import PCA

In [28]:
pca=PCA(n_components=500)
pca.fit(bowvec)
bowvec=pca.transform(bowvec)

In [29]:
bowvec.shape

(15404, 500)

In [30]:
from sklearn.model_selection import train_test_split
sent_train,sent_test,label_train,label_test=train_test_split(bowvec,lab_onehot,test_size=0.2)

In [31]:
from sklearn.linear_model import LinearRegression

In [32]:
lm=LinearRegression()

In [33]:
lm.fit(sent_train,label_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [34]:
preds=lm.predict(sent_test)

In [35]:
preds.shape

(3081, 736)

In [36]:
correct=0
for i in range(len(preds)):
    k=np.argmax(preds[i])
    if label_test[i][k]==1:
        correct+=1
print(100*correct/len(preds))

64.07010710808179


In [37]:
from sklearn.metrics import mean_squared_error
print(mean_squared_error(preds,label_test))

0.002260084773006754


In [38]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms

In [39]:
class customdataloader(torch.utils.data.Dataset):
    def __init__(self,sent,lab):
        self.sent=sent
        self.lab=lab
        
    def __len__(self):
        return len(self.lab)
    def __getitem__(self,idx):
        label=self.lab[idx]
        sentence=self.sent[idx]
        
        return sentence,label

In [40]:
import torch.nn.functional as F

In [41]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.fc1=nn.Linear(500,100)
        self.fc2=nn.Linear(100,736)
        self.dropout=nn.Dropout(0.2)
    def forward(self,x):
        l1=self.fc1(x)
        al1=F.relu(l1)
        al1=self.dropout(al1)
        l2=self.fc2(al1)
        return l2

In [42]:
def train(model,use_cuda,train_loader,optimizer,epoch):
    model.train()
    for batchid,(data,target) in enumerate(train_loader):
        y_onehot=target.argmax(dim=1,keepdim=True)
        y_onehot=torch.flatten(y_onehot)
        if use_cuda:
            data,y_onehot=data.cuda(),y_onehot.cuda()
        optimizer.zero_grad()
        output=model(data)
        loss=F.cross_entropy(output,y_onehot)
        loss.backward()
        optimizer.step()
        if batchid % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batchid * len(data), len(train_loader.dataset),
            100. * batchid / len(train_loader), loss.item()))

In [43]:
def test(model,use_cuda,test_loader):
    model.eval()
    test_loss=0
    correct=0
    with torch.no_grad():
        for data,target in test_loader:
            y_onehot=target
            if use_cuda:
                data,y_onehot=data.cuda(),y_onehot.cuda()
            output=model(data)
            #test_loss+=torch.sum((output-y_onehot)**2)
            pred=output.argmax(dim=1,keepdim=True)
            for i in range(len(pred)):
                if target[i][pred[i]]==1:
                    correct+=1
        #test_loss/=len(test_loader.dataset)
        print(100*correct/len(test_loader.dataset))

In [44]:
def seed(seed_value):
    torch.cuda.manual_seed_all(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed(seed_value)
    np.random.seed(seed_value)
    random.seed(seed_value)
    torch.backends.cudnn.benchmark=False
    torch.backends.cudnn.deterministic=True

In [45]:
def main():
    use_cuda=False
    seed(0)
    data1=customdataloader(sent_train,label_train)
    data2=customdataloader(sent_test,label_test)
    train_loader=torch.utils.data.DataLoader(data1,num_workers=0,batch_size=30,shuffle=True)
    test_loader=torch.utils.data.DataLoader(data2,num_workers=0,batch_size=50,shuffle=False)
    model=Net()
    if use_cuda:
        model=model.cuda()
    optimizer=optim.Adam(model.parameters(),lr=0.001)
    for epoch in range(1,26):
        train(model,use_cuda,train_loader,optimizer,epoch)
        test(model,use_cuda,test_loader)

In [46]:
import random
if __name__=="__main__":
    main()

Train Epoch: 1 [0/12323 (0%)]	Loss: 6.601882
Train Epoch: 1 [3000/12323 (24%)]	Loss: 5.607042
Train Epoch: 1 [6000/12323 (49%)]	Loss: 4.998100
Train Epoch: 1 [9000/12323 (73%)]	Loss: 4.749245
Train Epoch: 1 [12000/12323 (97%)]	Loss: 5.115880
18.597857838364167
Train Epoch: 2 [0/12323 (0%)]	Loss: 4.510907
Train Epoch: 2 [3000/12323 (24%)]	Loss: 5.236011
Train Epoch: 2 [6000/12323 (49%)]	Loss: 4.144692
Train Epoch: 2 [9000/12323 (73%)]	Loss: 4.049014
Train Epoch: 2 [12000/12323 (97%)]	Loss: 4.556823
27.26387536514119
Train Epoch: 3 [0/12323 (0%)]	Loss: 4.837703
Train Epoch: 3 [3000/12323 (24%)]	Loss: 3.209454
Train Epoch: 3 [6000/12323 (49%)]	Loss: 4.368441
Train Epoch: 3 [9000/12323 (73%)]	Loss: 4.381637
Train Epoch: 3 [12000/12323 (97%)]	Loss: 3.940663
36.384290814670564
Train Epoch: 4 [0/12323 (0%)]	Loss: 3.514222
Train Epoch: 4 [3000/12323 (24%)]	Loss: 3.222318
Train Epoch: 4 [6000/12323 (49%)]	Loss: 2.865823
Train Epoch: 4 [9000/12323 (73%)]	Loss: 3.146723
Train Epoch: 4 [12000/1232

In [47]:
import random
import nltk
from nltk.corpus import stopwords
import pandas as pd
import numpy as np

In [48]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Activation,Dropout

In [49]:
model=Sequential()
model.add(Dense(100,input_dim=500,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(736))

In [50]:
model.compile(optimizer='Adam',loss='binary_crossentropy')

In [51]:
model.fit(bowvec,lab_onehot,epochs=100)

Epoch 1/100
482/482 [==============================] - 3s 4ms/step - loss: 0.0357
Epoch 2/100
482/482 [==============================] - 2s 4ms/step - loss: 0.0333
Epoch 3/100
482/482 [==============================] - 2s 4ms/step - loss: 0.0325
Epoch 4/100
482/482 [==============================] - 2s 4ms/step - loss: 0.0321
Epoch 5/100
482/482 [==============================] - 2s 4ms/step - loss: 0.0319
Epoch 6/100
482/482 [==============================] - 2s 4ms/step - loss: 0.0318
Epoch 7/100
482/482 [==============================] - 2s 4ms/step - loss: 0.0316
Epoch 8/100
482/482 [==============================] - 2s 4ms/step - loss: 0.0315
Epoch 9/100
482/482 [==============================] - 2s 4ms/step - loss: 0.0314
Epoch 10/100
482/482 [==============================] - 2s 4ms/step - loss: 0.0313
Epoch 11/100
482/482 [==============================] - 2s 4ms/step - loss: 0.0313
Epoch 12/100
482/482 [==============================] - 2s 4ms/step - loss: 0.0313
Epoch 13/100


482/482 [==============================] - 2s 4ms/step - loss: 0.0297
Epoch 100/100
482/482 [==============================] - 2s 4ms/step - loss: 0.0297


In [52]:
score=model.evaluate(sent_train,label_train,verbose=0)

In [53]:
pred=model.predict(sent_test)

In [54]:
preds

array([[-1.00676585e-02,  3.80549245e-02, -8.22723359e-02, ...,
        -2.61734880e-04, -1.15586317e-03,  5.01880073e-04],
       [-3.26074660e-02,  3.62824202e-02,  1.44971579e-01, ...,
         1.56622170e-03, -9.91447596e-04, -2.21733644e-04],
       [-1.25031639e-02,  1.34501047e-03, -6.34720922e-03, ...,
         6.21292333e-04, -7.07354688e-04, -6.45473629e-05],
       ...,
       [ 6.19833291e-01, -9.56396945e-03, -5.72663248e-02, ...,
        -1.52812665e-03,  9.09248367e-04, -4.09506669e-04],
       [-9.31897946e-03, -1.45809595e-02,  1.83034614e-02, ...,
        -3.53974698e-04, -4.37907525e-04, -1.16585952e-03],
       [ 3.15130986e-02, -2.47604940e-02,  9.31406021e-02, ...,
        -1.22663751e-03,  1.00484816e-03, -1.38058036e-04]], dtype=float32)

In [55]:
preds=[]
for i in range(len(pred)):
    preds.append(pred[i].argmax())

In [56]:
label_test
correct=0
for i in range(len(label_test)):
    if label_test[i][preds[i]]==1:
        correct+=1
print(correct)
print(100*correct/len(label_test))

1753
56.897111327491075
